In [23]:
import os
import sys
import time
import logging
import subprocess
import signal
import argparse
import random
from datetime import datetime
from IPython.display import display, HTML, clear_output

# Set up logging directory - using absolute paths to avoid any issues
script_dir = os.path.dirname(os.path.abspath('__file__'))
log_dir = os.path.join(script_dir, 'output', 'logs')
os.makedirs(log_dir, exist_ok=True)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir, "trader.log")),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

def run_price_monitor(runtime=120, port=7497):
    """
    Run the price monitor script with specified runtime
    
    Parameters:
    runtime (int): How long to run the price monitor (in seconds)
    port (int): IBKR port to connect to
    
    Returns:
    bool: True if successful, False otherwise
    """
    logger.info(f"Starting price monitor with runtime: {runtime} seconds...")
    print(f"Starting price monitor with runtime: {runtime} seconds...")
    
    # Path to price_monitor.py using absolute path
    script_path = os.path.join(script_dir, 'price_monitor.py')
    
    if not os.path.exists(script_path):
        logger.error(f"Price monitor script not found at: {script_path}")
        print(f"ERROR: Price monitor script not found at: {script_path}")
        return False
    
    try:
        # Run price_monitor with specified runtime and port
        cmd = [sys.executable, script_path, '--runtime', str(runtime), '--port', str(port)]
        
        # For notebook, we'll use Popen and communicate to capture output
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        
        # Stream the output
        for line in iter(process.stdout.readline, ''):
            if not line:
                break
            print(line.strip())
        
        # Wait for process to complete with timeout
        try:
            exit_code = process.wait(timeout=runtime+10)
            if exit_code == 0:
                logger.info("Price monitor completed successfully")
                print("Price monitor completed successfully")
                
                # Wait a moment to ensure any database operations complete
                time.sleep(3)
                return True
            else:
                logger.error(f"Price monitor exited with code {exit_code}")
                print(f"ERROR: Price monitor exited with code {exit_code}")
                return False
        except subprocess.TimeoutExpired:
            process.kill()
            logger.error("Price monitor timed out")
            print("ERROR: Price monitor timed out")
            return False
    
    except Exception as e:
        logger.error(f"Error running price monitor: {str(e)}")
        print(f"ERROR: Error running price monitor: {str(e)}")
        return False

def run_order_placement(port=7497, allow_market_closed=False):
    """
    Run the order placement script
    
    Parameters:
    port (int): IBKR port to connect to
    allow_market_closed (bool): Whether to allow orders when market is closed
    
    Returns:
    bool: True if successful, False otherwise
    """
    logger.info("Starting order placement...")
    print("Starting order placement...")
    
    # Path to vertical_spread_order.py using absolute path
    script_path = os.path.join(script_dir, 'vertical_spread_order.py')
    
    if not os.path.exists(script_path):
        logger.error(f"Order placement script not found at: {script_path}")
        print(f"ERROR: Order placement script not found at: {script_path}")
        return False
    
    try:
        # Generate a random client ID to avoid connection conflicts
        client_id = random.randint(100, 9999)
        
        # Set up command with appropriate arguments
        cmd = [
            sys.executable, 
            script_path, 
            '--client', str(client_id), 
            '--port', str(port)
        ]
        
        # Add allow-market-closed flag if specified
        if allow_market_closed:
            cmd.append('--allow-market-closed')
        
        # For notebook, we'll use Popen and communicate to capture output
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        
        # Stream the output
        for line in iter(process.stdout.readline, ''):
            if not line:
                break
            print(line.strip())
        
        # Wait for process to complete with timeout
        try:
            exit_code = process.wait(timeout=60)  # 60 second timeout
            if exit_code == 0:
                logger.info("Order placement completed successfully")
                print("Order placement completed successfully")
                return True
            else:
                logger.error(f"Order placement exited with code {exit_code}")
                print(f"ERROR: Order placement exited with code {exit_code}")
                return False
        except subprocess.TimeoutExpired:
            process.kill()
            logger.error("Order placement timed out")
            print("ERROR: Order placement timed out")
            return False
    
    except Exception as e:
        logger.error(f"Error running order placement: {str(e)}")
        print(f"ERROR: Error running order placement: {str(e)}")
        return False

def run_trading_system(runtime=60, cycles=1, port=7497, allow_market_closed=False, interval=60):
    """
    Run the full trading system for specified number of cycles
    
    Parameters:
    runtime (int): Runtime for price monitor in seconds
    cycles (int): Number of trading cycles to run
    port (int): IBKR port to connect to
    allow_market_closed (bool): Whether to allow orders when markets are closed
    interval (int): Seconds to wait between cycle starts
    """
    
    logger.info("=" * 80)
    logger.info(f"Starting automated trading system (port: {port}, runtime: {runtime}s)")
    logger.info("=" * 80)

    display(HTML(f"<h3>Starting automated trading system</h3>"))
    display(HTML(f"<p>Port: {port} ({'Paper Trading' if port == 7497 else 'Live Trading'})</p>"))
    display(HTML(f"<p>Monitor runtime: {runtime} seconds</p>"))
    display(HTML(f"<p>Market closed orders: {'Allowed' if allow_market_closed else 'Not allowed'}</p>"))

    cycle_count = 0

    # Main loop
    while cycle_count < cycles:
        cycle_count += 1
        logger.info(f"Starting cycle {cycle_count} of {cycles}")
        display(HTML(f"<h4>Starting cycle {cycle_count} of {cycles}</h4>"))
        
        cycle_start = time.time()
        
        # Step 1: Run price monitor
        display(HTML("<p><b>Step 1:</b> Running price monitor...</p>"))
        monitor_success = run_price_monitor(runtime=runtime, port=port)
        
        # Step 2: Run order placement if monitoring was successful
        if monitor_success:
            display(HTML("<p><b>Step 2:</b> Running order placement...</p>"))
            order_success = run_order_placement(port=port, allow_market_closed=allow_market_closed)
            if not order_success:
                display(HTML("<p style='color:red'>Warning: Order placement failed or was incomplete</p>"))
                logger.warning("Order placement failed or was incomplete")
        else:
            display(HTML("<p style='color:red'>Error: Price monitoring failed, skipping order placement</p>"))
            logger.error("Price monitoring failed, skipping order placement")
        
        # Calculate how long to wait until next cycle
        cycle_duration = time.time() - cycle_start
        wait_time = max(0, interval - cycle_duration)
        
        # Wait for interval before starting the next cycle
        if cycle_count < cycles and wait_time > 0:
            display(HTML(f"<p>Cycle {cycle_count} complete. Waiting {wait_time:.1f} seconds until next cycle...</p>"))
            logger.info(f"Waiting {wait_time:.1f} seconds until next cycle...")
            
            # Use a progress indicator for waiting
            start_wait = time.time()
            while time.time() - start_wait < wait_time:
                elapsed = time.time() - start_wait
                percent = min(100, (elapsed / wait_time) * 100)
                
                # Clear the output and show progress
                clear_output(wait=True)
                display(HTML(f"<p>Cycle {cycle_count} complete. Waiting for next cycle...</p>"))
                display(HTML(f"<p>Progress: {percent:.1f}% ({elapsed:.1f}/{wait_time:.1f} seconds)</p>"))
                display(HTML(f"<div style='width:100%; background-color:#f0f0f0; height:20px'>"
                            f"<div style='width:{percent}%; background-color:#4CAF50; height:20px'></div></div>"))
                
                # Sleep for a short interval
                time.sleep(0.5)

    display(HTML(f"<h3>Trading system completed after {cycle_count} cycles</h3>"))
    logger.info(f"Trading system completed after {cycle_count} cycles")
    
    return cycle_count

# Display a message to confirm code loaded
print(f"Trading system functions loaded. Log file will be saved to: {os.path.join(log_dir, 'trader.log')}")

Trading system functions loaded. Log file will be saved to: /home/chris_s_dodd/source/optcom/monitor/output/logs/trader.log


In [ ]:
# Set your parameters here
runtime = 120               # Runtime for price monitor in seconds
cycles =  700               # Number of trading cycles to run
port = 7497                # IBKR port (7497 for paper, 7496 for live)
allow_market_closed = True # Allow orders when markets are closed
interval = 60              # Seconds to wait between cycle starts

# Display configuration
print("=== Trading System Configuration ===")
print(f"Price monitor runtime: {runtime} seconds")
print(f"Number of cycles: {cycles}")
print(f"IBKR port: {port} ({'Paper Trading' if port == 7497 else 'Live Trading'})")
print(f"Allow market closed orders: {allow_market_closed}")
print(f"Interval between cycles: {interval} seconds")
print("=================================")

# Run the trading system
run_trading_system(
    runtime=runtime,
    cycles=cycles,
    port=port,
    allow_market_closed=allow_market_closed,
    interval=interval
)

2025-06-04 13:18:17,733 - INFO - ================================================================================
2025-06-04 13:18:17,740 - INFO - Starting automated trading system (port: 7497, runtime: 120s)
2025-06-04 13:18:17,741 - INFO - ================================================================================


=== Trading System Configuration ===
Price monitor runtime: 120 seconds
Number of cycles: 700
IBKR port: 7497 (Paper Trading)
Allow market closed orders: True
Interval between cycles: 60 seconds


2025-06-04 13:18:17,751 - INFO - Starting cycle 1 of 700


2025-06-04 13:18:17,760 - INFO - Starting price monitor with runtime: 120 seconds...


Starting price monitor with runtime: 120 seconds...
/home/chris_s_dodd/source/optcom/monitor/price_monitor.py:2: DeprecationWarning:
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466

import pandas as pd
2025-06-04 13:18:18,759 - INFO - Connecting to IBKR at 127.0.0.1:7497
2025-06-04 13:18:18,772 - INFO - sent startApi
2025-06-04 13:18:18,773 - INFO - REQUEST startApi {}
2025-06-04 13:18:18,773 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x001\x00\x00'
2025-06-04 13:18:18,773 - INFO - ANSWER connectAck {}
2025-06-04 13:18:18,781 - INFO - ANSWER managedAccounts {'accountsList': 'DU9233079'}
2025-06-04 13:18:18,844 - INFO - ANSWER nextValidId {'ord

...

Starting order placement...
/home/chris_s_dodd/source/optcom/monitor/vertical_spread_order.py:5: DeprecationWarning:
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466

import pandas as pd
2025-06-04 16:58:52,314 - INFO - Processing strategies for date: 2025-06-04
2025-06-04 16:58:52,315 - INFO - Market closed orders allowed: True
2025-06-04 16:58:52,318 - INFO - Getting strategies for date: 2025-06-04
2025-06-04 16:58:52,323 - INFO - No strategies found for 2025-06-04


2025-06-04 16:58:52,442 - INFO - Order placement completed successfully
2025-06-04 16:58:52,444 - INFO - Starting cycle 89 of 700


Order placement completed successfully


2025-06-04 16:58:52,450 - INFO - Starting price monitor with runtime: 120 seconds...


Starting price monitor with runtime: 120 seconds...
/home/chris_s_dodd/source/optcom/monitor/price_monitor.py:2: DeprecationWarning:
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466

import pandas as pd
2025-06-04 16:58:53,350 - INFO - Connecting to IBKR at 127.0.0.1:7497
2025-06-04 16:58:53,381 - INFO - sent startApi
2025-06-04 16:58:53,382 - INFO - REQUEST startApi {}
2025-06-04 16:58:53,382 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x001\x00\x00'
2025-06-04 16:58:53,382 - INFO - ANSWER connectAck {}
2025-06-04 16:58:53,404 - INFO - ANSWER nextValidId {'orderId': 1}
2025-06-04 16:58:53,404 - INFO - Connected to IBKR
2025-06-04 16:58:53,405 -